In [148]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score, train_test_split, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np



In [149]:
# Завантаження даних
file_path = 'titanic.csv'
titanic_data = pd.read_csv(file_path)

In [150]:
# Вибір основних ознак (без нових ознак)
selected_columns = ['Survived', 'Pclass', 'Sex', 'Age', 'Fare']
titanic_selected = titanic_data[selected_columns]

In [151]:
# Обробка пропущених значень
titanic_selected['Age'].fillna(titanic_selected['Age'].median(), inplace=True)
titanic_selected['Fare'].fillna(titanic_selected['Fare'].median(), inplace=True)


C:\Users\User\AppData\Local\Temp\ipykernel_12984\2968045514.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_selected['Age'].fillna(titanic_selected['Age'].median(), inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12984\2968045514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_selected['Age'].fillna(titanic_sel

In [152]:
# Замінити категоріальні змінні на числові (Sex)
titanic_selected['Sex'] = titanic_selected['Sex'].map({'male': 0, 'female': 1})

C:\Users\User\AppData\Local\Temp\ipykernel_12984\1967952041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_selected['Sex'] = titanic_selected['Sex'].map({'male': 0, 'female': 1})


In [153]:

# Розділення на ознаки та цільову змінну
X = titanic_selected.drop('Survived', axis=1)
y = titanic_selected['Survived']


In [154]:
# Стандартизація числових ознак
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [155]:
# Розділення даних на навчальні та тестові
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [156]:
# Масштабування даних
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [157]:
# 1. Використовуємо стратифіковану крос-валідацію
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [158]:
# 2. Налаштування моделей і GridSearchCV
classifiers = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(probability=True)
}

param_grids = {
    'Decision Tree': {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]},
    'Random Forest': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10], 'min_samples_split': [2, 5]},
    'SVM': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
}


In [159]:
# 3. Виконання крос-валідації та пошук найкращих параметрів
best_models = {}
for name, model in classifiers.items():
    grid_search = GridSearchCV(estimator=model, param_grid=param_grids[name], cv=skf, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Найкращі параметри для {name}: {grid_search.best_params_}")
    print(f"Найкращий результат для {name}: {grid_search.best_score_:.4f}")


Найкращі параметри для Decision Tree: {'max_depth': 3, 'min_samples_split': 2}
Найкращий результат для Decision Tree: 1.0000
Найкращі параметри для Random Forest: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50}
Найкращий результат для Random Forest: 1.0000
Найкращі параметри для SVM: {'C': 0.1, 'kernel': 'linear'}
Найкращий результат для SVM: 1.0000


In [160]:
# 4. Оцінка моделей на тестовій вибірці
for name, model in best_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\nОцінка моделі: {name}")
    print(f"Точність на тестовій вибірці: {accuracy_score(y_test, y_pred):.4f}")
    print(f"F1 Score на тестовій вибірці: {f1_score(y_test, y_pred):.4f}")
    print(f"ROC AUC Score на тестовій вибірці: {roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]):.4f}")



Оцінка моделі: Decision Tree
Точність на тестовій вибірці: 1.0000
F1 Score на тестовій вибірці: 1.0000
ROC AUC Score на тестовій вибірці: 1.0000

Оцінка моделі: Random Forest
Точність на тестовій вибірці: 1.0000
F1 Score на тестовій вибірці: 1.0000
ROC AUC Score на тестовій вибірці: 1.0000

Оцінка моделі: SVM
Точність на тестовій вибірці: 1.0000
F1 Score на тестовій вибірці: 1.0000
ROC AUC Score на тестовій вибірці: 1.0000


In [161]:
# 5. Оцінка моделей за допомогою крос-валідації
for name, model in best_models.items():
    y_pred = cross_val_predict(model, X_scaled, y, cv=skf, method='predict')
    print(f"\nОцінка моделі: {name}")
    print(f"Середня точність крос-валідації: {np.mean(cross_val_score(model, X_scaled, y, cv=skf, scoring='accuracy')):.4f}")
    print(f"Середнє F1 Score: {f1_score(y, y_pred):.4f}")
    print(f"ROC AUC Score: {roc_auc_score(y, cross_val_predict(model, X_scaled, y, cv=skf, method='predict_proba')[:, 1]):.4f}")



Оцінка моделі: Decision Tree
Середня точність крос-валідації: 1.0000
Середнє F1 Score: 1.0000
ROC AUC Score: 1.0000

Оцінка моделі: Random Forest
Середня точність крос-валідації: 1.0000
Середнє F1 Score: 1.0000
ROC AUC Score: 1.0000

Оцінка моделі: SVM
Середня точність крос-валідації: 1.0000
Середнє F1 Score: 1.0000
ROC AUC Score: 1.0000


In [162]:
# 6. Прогноз для 10 нових випадків
new_data = X_test[:10]  # Використовуємо перші 10 випадкових рядків з тестових даних
for name, model in best_models.items():
    predictions = model.predict(new_data)
    print(f"\nПрогноз для моделі {name} на 10 випадків: {predictions}")



Прогноз для моделі Decision Tree на 10 випадків: [0 1 0 0 1 0 1 0 1 0]

Прогноз для моделі Random Forest на 10 випадків: [0 1 0 0 1 0 1 0 1 0]

Прогноз для моделі SVM на 10 випадків: [0 1 0 0 1 0 1 0 1 0]


Усі моделі продемонстрували ідеальну точність та F1 Score під час крос-валідації. Прогнози для 10 нових випадків були однакові для всіх моделей. Це може свідчити про те, що всі три моделі добре узгоджуються з наявними даними або ж є ймовірність перетренування.